# 面板

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as st 

df1= pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\城市群2.xlsx',sheet_name='iu')
df11=df1.melt(id_vars=['city'],var_name='year',value_name='iu')
#df1.head()
df2= pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\城市群2.xlsx',sheet_name='gov')
df21=df2.melt(id_vars=['city'],var_name='year',value_name='gov')
df3= pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\城市群2.xlsx',sheet_name='fdi')
df31=df3.melt(id_vars=['city'],var_name='year',value_name='fdi')
df4= pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\城市群2.xlsx',sheet_name='pop')
df41=df4.melt(id_vars=['city'],var_name='year',value_name='pop')
df5= pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\城市群2.xlsx',sheet_name='gdp')
df51=df5.melt(id_vars=['city'],var_name='year',value_name='gdp')
df6= pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\城市群2.xlsx',sheet_name='teans')
df61=df6.melt(id_vars=['city'],var_name='year',value_name='teans')
df7= pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\城市群2.xlsx',sheet_name='edu')
df71=df7.melt(id_vars=['city'],var_name='year',value_name='edu')
df8= pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\城市群2.xlsx',sheet_name='m')
df81=df8.melt(id_vars=['city'],var_name='year',value_name='m')
#提取列
df12=df11.iloc[:,2]
df22=df21.iloc[:,2]
df32=df31.iloc[:,2]
df42=df41.iloc[:,2]
df52=df51.iloc[:,2]
df62=df61.iloc[:,2]
df72=df71.iloc[:,2]
df82=df81.iloc[:,2]

df= pd.concat([df11,df22,df32,df42,df52,df62,df72,df82],axis=1)
df
df.to_excel('D:\\学校\\毕业论文\\数据\\城市群\\总城市群面板.xlsx',index=False)



# 缺失情况

In [8]:
data = pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\总城市群面板1.xlsx',sheet_name=0)
data
# 将空值形式的缺失值转换成可识别的类型
data = data.replace(' ', np.NaN)
#data = data.replace('0', np.NaN)
print(data.columns)
#将每列中缺失值的个数统计出来
null_all = data.isnull().sum()
null_all
data = pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\总城市群面板1.xlsx',sheet_name=0)
data.isna().mean()

Index(['city', 'year', 'iu', 'gov', 'fdi', 'pop', 'gdp', 'teans', 'edu', 'm'], dtype='object')


city     0.000000
year     0.000000
iu       0.002674
gov      0.001783
fdi      0.005348
pop      0.001783
gdp      0.003119
teans    0.026738
edu      0.012923
m        0.002674
dtype: float64

# 插补

In [13]:
#定义随机森林算法补齐函数
# data_df: DataFrame类型的数据#前面已经删除
# obj_column：待填补缺失值的列名
# missing_other_column：数据中含义空值的其他列
########## 缺失值处理
data = pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\总城市群面板1.xlsx',sheet_name=0)
data1=data.iloc[:,2:]
#data1 = data1.replace('', np.NaN)
data1

,iu,gov,fdi,pop,gdp,teans,edu,m
0,1.717397,0.259630,0.586473,683.820000,3261,7.354439,392.393642,8.877473
1,0.893475,0.127518,0.667270,776.850000,26532,12.457444,264.888553,3.476170
2,0.789845,0.582225,0.184434,574.530000,15188,4.549281,24.658927,2.419258
3,0.545797,0.563550,0.155933,524.260000,18387,4.119951,112.516192,1.385234
4,1.268282,0.785688,0.399129,363.270000,14236,6.568493,23.378415,2.394962
...,...,...,...,...,...,...,...,...
2239,0.869386,0.183946,0.456446,48.934275,5338,1.121787,277.333333,2.349930
2240,0.654469,0.157473,0.498561,28.643815,595,3.666667,133.238952,2.237398
2241,1.212537,0.255427,0.468818,43.532816,34481,3.135850,31.154717,2.949930
2242,1.513269,0.348140,0.112789,23.533488,44794,7.759494,53.645570,2.369495


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def fill_miss_byRandomForest(data_df , obj_column ):
    
    # 分成已知该特征和未知该特征两部分
    known = data_df[data_df[obj_column].notnull()]
    unknown = data_df[data_df[obj_column].isnull()]
    # y为结果标签值
    y_know = known[obj_column]
    # X为特征属性值
    X_know= known.drop(obj_column , axis = 1)
    from sklearn.ensemble import RandomForestRegressor
    rfr = RandomForestRegressor(random_state=0, n_estimators=200,max_depth=3,n_jobs=-1)
    rfr.fit(X_know,y_know)
    # 用得到的模型进行未知特征值预测
    # X为特征属性值
    X_unknow= unknown.drop(obj_column , axis = 1)
    predicted = rfr.predict(X_unknow).round(0)
    data_df.loc[(data_df[obj_column].isnull()), obj_column] = predicted
    return data_df

In [15]:
data11=fill_miss_byRandomForest(data1, 'iu')#用随机森林填补MonthlyIncome 的缺失值
#data.shape

ValueError: could not convert string to float: ''

In [18]:
# 缺失值处理：补充缺失的数据
# 三种方法：Lagrange插值法和Newton插值法以及Series自带的interpolate
#1、Lagrange插值法和Newton插值法解决实际问题中关于只提供复杂的离散数据的函数求值问题，通过将所考察的函数简单化，构造关于离散数据实际函数f（x）的近似函数P（x），从而可以计算未知点出的函数值，是插值法的基本思路。
#2、实际上Lagrange插值法和Newton插值法是同一种方法的两种变形，其构造拟合函数的思路是相同的，而实验中两个实际问题用两种算法计算出结果是相同的。
#3、实验所得结果精确度并不高，一方面是因为所给数据较少，另一方面也是主要方面在Win32中C++中数据类型double精度只有7位，计算机在进行浮点运算时截断运算会导致误差。实际问题中，测量数据也可能导致误差。
#4、在解决实际问题中，更多是利用精确且高效的计算机求解。所以解决问题时不仅要构造可求解的算法，更重要是构造合理的可以编写成程序由计算机求解的算法，而算法的优化不仅可以节省时间空间，更能得到更为精确有价值的结果。
# <1> 拉格朗日插值法  
# 自定义列向量插值函数  
# s为列向量， n为被插值的位置， k为取前后的数据个数， 默认为5  
data0 = pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\总城市群面板1.xlsx',sheet_name=0)
data0=data.iloc[:,2:]

from scipy.interpolate import lagrange #导入拉格朗日插值函数  
  
def plotinterplate_columns(s, n, k=5):  
    y = s[list(range(n-k,n)) + list(range(n+1, n+1+k))]  
    y = y[y.notnull()]#剔除空值  
    return lagrange(y.index, list(y))(n)#向位置n出插值并返回该插值结果  y.index返回的是费缺失值在原来列表中的位置

In [21]:
# 逐个判断每列是否需要插值  
lagij = []  
for i in data0.columns:  
    for j in range(len(data1)):  
        if (data0[i].isnull())[j]:  
            data0[i][j] = plotinterplate_columns(data0[i],j)  
            lagij.append((i,j,data0[i][j]))  
              
print(data0)  
data0.to_excel('D:\\学校\\毕业论文\\数据\\城市群\\拉插值总.xlsx',index=False)


            iu       gov       fdi         pop    gdp      teans         edu  \
0     1.717397  0.259630  0.586473  683.820000   3261   7.354439  392.393642   
1     0.893475  0.127518  0.667270  776.850000  26532  12.457444  264.888553   
2     0.789845  0.582225  0.184434  574.530000  15188   4.549281   24.658927   
3     0.545797  0.563550  0.155933  524.260000  18387   4.119951  112.516192   
4     1.268282  0.785688  0.399129  363.270000  14236   6.568493   23.378415   
...        ...       ...       ...         ...    ...        ...         ...   
2239  0.869386  0.183946  0.456446   48.934275   5338   1.121787  277.333333   
2240  0.654469  0.157473  0.498561   28.643815    595   3.666667  133.238952   
2241  1.212537  0.255427  0.468818   43.532816  34481   3.135850   31.154717   
2242  1.513269  0.348140  0.112789   23.533488  44794   7.759494   53.645570   
2243  0.946924  0.356716  0.646595  129.587394  35181   1.400000    7.400000   

             m  
0     8.877473  
1    

In [25]:

 
 
# <2> 牛顿插值法  
  
import matplotlib.pyplot as plt  
  
""" 
@brief:   计算n阶差商 f[x0, x1, x2 ... xn]  
@param:   xi   所有插值节点的横坐标集合                                                        o 
@param:   fi   所有插值节点的纵坐标集合                                                      /   \ 
@return:  返回xi的i阶差商(i为xi长度减1)                                                     o     o 
@notice:  a. 必须确保xi与fi长度相等                                                        / \   / \ 
          b. 由于用到了递归，所以留意不要爆栈了.                                           o   o o   o 
          c. 递归减递归(每层递归包含两个递归函数), 每层递归次数呈二次幂增长，总次数是一个满二叉树的所有节点数量(所以极易栈溢出)                                                                                      
"""  
  
def get_order_diff_quot(xi = [], fi = []):  
    if len(xi) > 2 and len(fi) > 2:  
        return (get_order_diff_quot(xi[:len(xi) - 1], fi[:len(fi) - 1]) - get_order_diff_quot(xi[1:len(xi)], fi[1:len(fi)])) / float(xi[0] - xi[-1])  
    return (fi[0] - fi[1]) / float(xi[0] - xi[1])  
""" 
 
@brief:  获得Wi(x)函数; 
         Wi的含义举例 W1 = (x - x0); W2 = (x - x0)(x - x1); W3 = (x - x0)(x - x1)(x - x2) 
@param:  i  i阶(i次多项式) 
@param:  xi  所有插值节点的横坐标集合 
@return: 返回Wi(x)函数 
"""  
  
def get_Wi(i = 0, xi = []):  
    def Wi(x):  
        result = 1.0  
        for each in range(i):  
            result *= (x - xi[each])  
        return result  
    return Wi  
""" 
@brief: 获得牛顿插值函数 
"""  
  
def get_Newton_inter(xi = [], fi = []):  
    def Newton_inter(x):  
        result = fi[0]  
        for i in range(2, len(xi)):  
            result += (get_order_diff_quot(xi[:i], fi[:i]) * get_Wi(i-1, xi)(x))  
        return result  
    return Newton_inter  
# 自定义列向量插值函数  
# s为列向量， n为被插值的位置，k为取前后的数据个数， 默认为5  
def plotnewton_columns(s):  
    y = s[s.notnull()]#剔除空值  
    Nx= get_Newton_inter(y.index, list(y))         #向位置n出插值并返回该插值结果  
    return Nx  
  



In [28]:
import pandas as pd  
from scipy.optimize import newton #导入牛顿插值函数  
data= pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\总城市群面板1.xlsx',sheet_name=0)
data0=data.iloc[:,2:]
data0.columns

Index(['iu', 'gov', 'fdi', 'pop', 'gdp', 'teans', 'edu', 'm'], dtype='object')

In [ ]:
newij = []  
for i in data0.columns:  
    Newton = plotnewton_columns(data0[i])  
    for j in range(len(data1)):  
        if (data0[i].isnull())[j]:  
            data0[i][j] = Newton(j)  
            newij.append((i,j,data0[i][j]))  
newij 

In [23]:
 
# <3> Series自带的iterplote  
data0 = pd.read_excel('D:\\学校\\毕业论文\\数据\\城市群\\总城市群面板1.xlsx',sheet_name=0)
data0=data.iloc[:,2:]

###利用Pandas中interpolate进行缺失值的补充  
data_out = data0.interpolate()  


In [24]:
df = data_out - data0.fillna(0)  

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
Serij = []  
for i in df.columns:  
    for j in range(len(df)):  
        if df[i][j] != 0:  
            Serij.append((i,j,df[i][j]))  
Serij

In [ ]:
#1、导入必要的库
import miceforest as mf # miceforest导入失败的，先去终端 pip install miceforest安装一下import pandas as pd
import numpy as npimport seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsclassifier ,KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor，RandomForestclassifierfrom sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegressionfrom sklearn.svm import SVR


In [ ]:
#sns.set ( font= 'SimHei " ,font_scale=1.5 )
#windows用户解决seaborn中文显示问
题
sns.set ( font= ' Arial Unicode Ms ' ,font_scale=0.7 )
#mac用户解决seaborn中文显示问题
plt.rcParams [ ' font.sans-serif ' ] = [ 'Arial Unicode MS ' ]
#用来正常显示中文标签
plt.rcParams [ " axes.unicode_minus ']= False
#用来正常显示负号
#2、导入数据
data = pd.read_csv ( " data/pima2.csv" )data. isnull( ) -sum( )#查看数据缺失值情况
